In [57]:
import json
import time
import numpy as np
from functools import wraps


class Platform:

    def __init__(self, config_json, table_path):
        self.table = None  # get table...

        with open(config_json, 'r') as f:
            config = json.load(f)

        self.algo_instance = config["AlgorithmClass"]
        self.preprocessing_type = config["PreprocessingClass"]
        self.run_type = config["runType"]
        self.logs = config["logs"]

        self.benchmark = {"name": self.algo_instance.__name__}

        if self.run_type == 'test':
            self.test()
        elif self.run_type == 'compress':
            self.compress()
        elif self.run_type == 'decompress':
            self.decompress()

    def measure_time(flag):
        def decorator(func):
            def wrapper(self, *args, **kwargs):
                start_time = time.time()
                result = func(self, *args, **kwargs)
                end_time = time.time()

                if getattr(self, flag):
                    print(f"Function {func.__name__} took {end_time - start_time} seconds to execute.")

                return result
            return wrapper
        return decorator

    @measure_time("logs")
    def compress(self):
        compressed_data = self.algo_instance.compress(np.array(self.table))
        if self.logs:
            compression_rate = len(compressed_data) / len(self.table)
            self.benchmark["compression_rate"] = compression_rate

    @measure_time("logs")
    def decompress(self, compressed_data):
        decompressed_data = self.algo_instance.decompression(compressed_data)
        if self.logs:
            loss_rate = None  # get_loss_rate(self.table, decompressed_data)
            self.benchmark["loss_rate"] = loss_rate

    @measure_time("logs")
    def test(self):
        compressed_data = self.compress()
        decompressed_data = self.decompress(compressed_data)

In [ ]:
import abc

class CompressionAlgorithm(abc.ABC):
    def __init__(self):
        ...
    @property
    @abc.abstractmethod
    def name(self):
        ...

    @abc.abstractmethod
    def compression(self, table):
        ...

    @abc.abstractmethod
    def decompression(self, compressed_data):
        ...

class LearningCompressionAlgorithm(CompressionAlgorithm):

    @abc.abstractmethod
    def fit(self, training_data):
        ...

class NonLearningCompressionAlgorithm(CompressionAlgorithm):
    ...

class LosslessCompressionAlgorithm(CompressionAlgorithm):
    ...

class LossyCompressionAlgorithm(CompressionAlgorithm):
    ...

In [ ]:
class DummyCompressionAlgorithm(LosslessCompressionAlgorithm):
    name = 'Dummy Compression Algorithm'
    def compression(self, table):
        time.sleep(2)
        return table
    def decompression(self, compressed_data):
        time.sleep(1)
        return compressed_data

In [ ]:
class Processing(abc.ABC):
    def __init__(self):
        ...
    @property
    @abc.abstractmethod
    def name(self):
        ...

    @abc.abstractmethod
    def preprocess(self, raw_table):
        ...

    @abc.abstractmethod
    def postprocess(self, processed_table):
        ...

In [ ]:
class DummyProcessing(Processing):
    name = "Dummy Pre/Post-Processing"
    def preprocess(self, raw_table):
        return raw_table
    def postprocess(self, processed_table):
        return processed_table


In [ ]:
import numpy as np
platform = Platform([DummyCompressionAlgorithm()], [np.array([1,2,3])])
platform.compress_table()
platform.results